In [14]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pandas as pd

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-21 19:06:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2021-04-21 19:06:17 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame where total_votes is greater than 20
vine_df_20 = vine_df.filter('total_votes > 20')
vine_df_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R2V9VX4YQ8TQ24|          5|           33|         36|   N|     

In [16]:
vine_df_20.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [22]:
vine_df_20 = vine_df_20.withColumn('percent', (vine_df_20['helpful_votes']  / vine_df_20['total_votes']))
vine_df_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|            percent|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|0.25806451612903225|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|                1.0|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|  0.855072463768116|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|0.21428571428571427|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|                1.0|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|            0.84375|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|               0.85|
| RRVJ7TYDEK7EO|    

In [24]:
# Filter vine_df_20 to create a new DataFrame that shows rows where helpful_votes/total_votes >= .5
helpful_reviews_df = vine_df_20.filter('percent >= .5')
helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|               1.0|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y| 0.855072463768116|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|               1.0|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|           0.84375|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|              0.85|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|               1.0|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|0.8769230769230769|
|R2V9VX4YQ8TQ24|          5|  

In [27]:
# Create a new dataframe from helpful_reviews_df of only vine reviews
vine_reviews_df = helpful_reviews_df.filter(helpful_reviews_df['vine'] == 'Y')
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| REKN4PLM5174P|          4|          219|        225|   Y|                N|0.9733333333333334|
| RQOG8IIJX7CKB|          4|           21|         27|   Y|                N|0.7777777777777778|
|R33EJ9PSZR13CM|          4|           26|         30|   Y|                N|0.8666666666666667|
|R37GSXLA11XF8N|          5|           30|         34|   Y|                N|0.8823529411764706|
|R2YE4F9ESKFWEK|          3|           20|         23|   Y|                N|0.8695652173913043|
|R1E4L7HHB8TBP9|          4|           80|         92|   Y|                N|0.8695652173913043|
|R2C2WXDXMZL005|          2|           20|         25|   Y|                N|               0.8|
|R1PPY4U0K03KTE|          5|  

In [28]:
# Creat a dataframe from helpful_reviews_df of only non-vine reviews
nonvine_reviews_df = helpful_reviews_df.filter(helpful_reviews_df['vine'] == 'N')
nonvine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|               1.0|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y| 0.855072463768116|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|               1.0|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|           0.84375|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|              0.85|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|               1.0|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|0.8769230769230769|
|R2V9VX4YQ8TQ24|          5|  

In [37]:
# Determine the total number of reviews, number of 5 star reviews and percentage of 5 star reviews for vine reviews
total_vine_reviews = vine_reviews_df.count()
star5_vine = vine_reviews_df.filter('star_rating = 5').count()
star5_percent = star5_vine / total_vine_reviews
print(total_vine_reviews, star5_vine, star5_percent)

76 30 0.39473684210526316


In [38]:
# Determine the total number of reviews, number of 5 star reviews and percentage of 5 star reviews for non-vine reviews
total_nonvine = nonvine_reviews_df.count()
star5_nonvine = nonvine_reviews_df.filter('star_rating = 5').count()
star5_nonvine_percent = star5_nonvine / total_nonvine
print(total_nonvine, star5_nonvine, star5_nonvine_percent)

23015 11914 0.5176623940908104
